# Load DATA

To load your EEG recording

In [3]:
import mne
mne.channels.get_builtin_montages()

['standard_1005',
 'standard_1020',
 'standard_alphabetic',
 'standard_postfixed',
 'standard_prefixed',
 'standard_primed',
 'biosemi16',
 'biosemi32',
 'biosemi64',
 'biosemi128',
 'biosemi160',
 'biosemi256',
 'easycap-M1',
 'easycap-M10',
 'EGI_256',
 'GSN-HydroCel-32',
 'GSN-HydroCel-64_1.0',
 'GSN-HydroCel-65_1.0',
 'GSN-HydroCel-128',
 'GSN-HydroCel-129',
 'GSN-HydroCel-256',
 'GSN-HydroCel-257',
 'mgh60',
 'mgh70',
 'artinis-octamon',
 'artinis-brite23',
 'brainproducts-RNP-BA-128']

In [ ]:
raw = mne.io.read_raw()
raw.set_montage("")

or download an exemple dataset

In [31]:
import mne
from mne.datasets import sample

data_path = sample.data_path()
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_filt-0-40_raw.fif'

raw = mne.io.read_raw_fif(raw_fname).crop(0, 60).pick(['eeg', 'ecg', 'eog']).load_data()
raw.filter(1, 30, fir_design='firwin')

Opening raw data file C:\Users\ferat\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
Removing projector <Projection | PCA-v1, active : False, n_channels : 102>
Removing projector <Projection | PCA-v2, active : False, n_channels : 102>
Removing projector <Projection | PCA-v3, active : False, n_channels : 102>
Reading 0 ... 9009  =      0.000 ...    59.999 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwi

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:    0.0s finished


<Raw | sample_audvis_filt-0-40_raw.fif, 61 x 9010 (60.0 s), ~7.1 MB, data loaded>

In [17]:
%matplotlib qt
mne.viz.set_browser_backend('qt')  
raw.plot()

Channels marked as bad:
['EEG 053']


## ICA

ICA is [`mne.preprocessing.ica`](https://mne.tools/stable/generated/mne.preprocessing.ICA.html?highlight=ica#mne.preprocessing.ICA)

In [1]:
from mne.preprocessing import ICA

ICA?

Init signature:
ICA(
    n_components=None,
    *,
    noise_cov=None,
    random_state=None,
    method='fastica',
    fit_params=None,
    max_iter='auto',
    allow_ref_meg=False,
    verbose=None,
)
Docstring:     
Data decomposition using Independent Component Analysis (ICA).

This object estimates independent components from :class:`mne.io.Raw`,
:class:`mne.Epochs`, or :class:`mne.Evoked` objects. Components can
optionally be removed (for artifact repair) prior to signal reconstruction.

.. warning:: ICA is sensitive to low-frequency drifts and therefore
             requires the data to be high-pass filtered prior to fitting.
             Typically, a cutoff frequency of 1 Hz is recommended.

Parameters
----------
n_components : int | float | None
    Number of principal components (from the pre-whitening PCA step) that
    are passed to the ICA algorithm during fitting:

    - :class:`int`
        Must be greater than 1 and less than or equal to the number of
        channels.


### n_components

In [2]:
n_components=59

### Method

`fastica`

Aapo Hyvärinen. Fast and robust fixed-point algorithms for independent component analysis. IEEE Transactions on Neural Networks, 10(3):626–634, 1999. doi:10.1109/72.761722.

`infomax`

Anthony J. Bell and Terrence J. Sejnowski. An information-maximization approach to blind separation and blind deconvolution. Neural Computation, 7(6):1129–1159, 1995. doi:10.1162/neco.1995.7.6.1129.

`infomax` + `fit_params=dict(extended=True)`

Te-Won Lee, Mark Girolami, and Terrence J. Sejnowski. Independent component analysis using an extended infomax algorithm for mixed subgaussian and supergaussian sources. Neural Computation, 11(2):417–441, 1999. doi:10.1162/089976699300016719.

`picard`

Pierre Ablin, Jean-Francois Cardoso, and Alexandre Gramfort. Faster Independent Component Analysis by preconditioning with hessian approximations. IEEE Transactions on Signal Processing, 66(15):4040–4049, 2018. doi:10.1109/TSP.2018.2844203.

In [3]:
ica = ICA(n_components=n_components,
          noise_cov=None,
          random_state=42,
          method='picard', # picard solver
          fit_params=dict(ortho=False, extended=True), # extended infomax
          max_iter='auto',
          allow_ref_meg=False, # MEG only
          verbose=None)

### Fit

In [4]:
ica.fit?

Signature:
ica.fit(
    inst,
    picks=None,
    start=None,
    stop=None,
    decim=None,
    reject=None,
    flat=None,
    tstep=2.0,
    reject_by_annotation=True,
    verbose=None,
)
Docstring:
Run the ICA decomposition on raw data.

Caveat! If supplying a noise covariance keep track of the projections
available in the cov, the raw or the epochs object. For example,
if you are interested in EOG or ECG artifacts, EOG and ECG projections
should be temporally removed before fitting the ICA.

Parameters
----------
inst : instance of Raw or Epochs
    The data to be decomposed.
picks : str | list | slice | None
    Channels to include. Slices and lists of integers will be interpreted as 
    channel indices. In lists, channel *type* strings (e.g., ``['meg', 
    'eeg']``) will pick channels of those types, channel *name* strings (e.g., 
    ``['MEG0111', 'MEG2623']`` will pick the given channels. Can also be the 
    string values "all" to pick all channels, or "data" to pick :term:

In [14]:
ica.fit(raw)

Fitting ICA to data using 59 channels (please be patient, this may take a while)
Selecting by number: 59 components
Fitting ICA took 11.4s.


Method,picard
Fit,329 iterations on raw data (9010 samples)
ICA components,59
Explained variance,100.0 %
Available PCA components,59
Channel types,eeg
ICA components marked for exclusion,—


### Selecting Bad components

In [18]:
ica.plot_components(inst=raw);

    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


...

C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


In [16]:
ica.plot_sources(inst=raw);

Creating RawArray with float64 data, n_channels=59, n_times=9010
    Range : 6450 ... 15459 =     42.956 ...   102.954 secs
Ready.


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


...

C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


    Using multitaper spectrum estimation with 7 DPSS windows
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated


C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\mne\viz\epochs.py:543: MatplotlibDeprecationWarning: The join function was deprecated in Matplotlib 3.6 and will be removed two minor releases later.
  ax['evoked'].get_shared_x_axes().join(ax['evoked'], ax_im)


### MNE ICA labels

Li et al., (2022). MNE-ICALabel: Automatically annotating ICA components with ICLabel in Python. Journal of Open Source Software, 7(76), 4484, https://doi.org/10.21105/joss.04484


In [19]:
!pip install mne-icalabel

     --------------------------------------- 10.8/10.8 MB 27.3 MB/s eta 0:00:00
     -------------------------------------- 161.8/161.8 MB 7.9 MB/s eta 0:00:00
  Using cached typing_extensions-4.3.0-py3-none-any.whl (25 kB)


In [20]:
from mne_icalabel import label_components

# assuming you have a Raw and ICA instance previously fitted
label_components(raw, ica, method='iclabel')

C:\Users\ferat\Miniconda3\envs\eeg39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\ferat\AppData\Local\Temp\ipykernel_1768\3350189304.py:4: RuntimeWarning: The provided Raw instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  label_components(raw, ica, method='iclabel')
C:\Users\ferat\AppData\Local\Temp\ipykernel_1768\3350189304.py:4: RuntimeWarning: The provided Raw instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  label_components(raw, ica, method='icla

{'y_pred_proba': array([0.9278839 , 0.7647001 , 0.9970324 , 0.9798221 , 0.5220299 ,
        0.9901547 , 0.9239477 , 0.52995855, 0.97791314, 0.96155393,
        0.99630463, 0.97647846, 0.9796555 , 0.7804446 , 0.5917051 ,
        0.70175546, 0.99670655, 0.5928928 , 0.55599785, 0.6810263 ,
        0.64349407, 0.88102883, 0.9994302 , 0.37597826, 0.8864524 ,
        0.56222326, 0.6092847 , 0.9232826 , 0.97671515, 0.62081695,
        0.46128488, 0.7829498 , 0.89938015, 0.8766513 , 0.48444182,
        0.5705153 , 0.5365608 , 0.62708753, 0.8230916 , 0.81559336,
        0.89698815, 0.4400778 , 0.6991721 , 0.79421175, 0.5745903 ,
        0.4575032 , 0.80572164, 0.56198573, 0.82850015, 0.79314744,
        0.56466633, 0.96095467, 0.6075463 , 0.7045741 , 0.5564039 ,
        0.5045259 , 0.45872688, 0.9676861 , 0.91065294], dtype=float32),
 'labels': ['eye blink',
  'brain',
  'brain',
  'brain',
  'heart beat',
  'brain',
  'brain',
  'brain',
  'brain',
  'brain',
  'brain',
  'brain',
  'brain',
 

### Apply

In [21]:
raw_ica = ica.apply(raw)

Applying ICA to Raw instance
    Transforming to ICA space (59 components)
    Zeroing out 0 ICA components
    Projecting back using 59 PCA components


In [22]:
raw_ica.plot()

Channels marked as bad:
['EEG 053']


### Additional Preprocessing steps

In [29]:
raw_ica.interpolate_bads()

Interpolating bad channels
    Automatic origin fit: head of radius 91.0 mm
Computing interpolation matrix from 59 sensor positions
Interpolating 1 sensors


<Raw | sample_audvis_filt-0-40_raw.fif, 60 x 9010 (60.0 s), ~7.1 MB, data loaded>

In [30]:
raw_ica.set_eeg_reference('average')

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Removing existing average EEG reference projection.


<Raw | sample_audvis_filt-0-40_raw.fif, 60 x 9010 (60.0 s), ~7.1 MB, data loaded>

### Save

#### ICA

In [ ]:
ica.save("my_ica-ica.fif")

#### EEG

In [27]:
mne.export.export_raw('raw_ica.vhdr', raw)

C:\Users\ferat\AppData\Local\Temp\ipykernel_1768\2533428595.py:1: RuntimeWarning: Raw instance has unapplied projectors. Consider applying them before exporting with raw.apply_proj().
  mne.export.export_raw('raw_ica.vhdr', raw)
